# Nonnegative Matrix Factorization

Nonnegative matrix factorization (NNMF) was introduced by [Lee and Seung (1999)](https://www.nature.com/articles/44565) as an analog of principal components and vector quantization with applications in data compression and clustering. In this homework we consider algorithms for fitting NNMF and (optionally) high performance computing using graphical processing units (GPUs).

<img src="./nnmf.png" width="500" align="center"/>

In mathematical terms, one approximates a data matrix $\mathbf{X} \in \mathbb{R}^{m \times n}$ with nonnegative entries $x_{ij}$ by a product of two low-rank matrices $\mathbf{V} \in \mathbb{R}^{m \times r}$ and $\mathbf{W} \in \mathbb{R}^{r \times n}$ with nonnegative entries $v_{ik}$ and $w_{kj}$. Consider minimization of the squared Frobenius norm
$$
	L(\mathbf{V}, \mathbf{W}) = \|\mathbf{X} - \mathbf{V} \mathbf{W}\|_{\text{F}}^2 = \sum_i \sum_j \left(x_{ij} - \sum_k v_{ik} w_{kj} \right)^2, \quad v_{ik} \ge 0, w_{kj} \ge 0,
$$
which should lead to a good factorization. Later in the course we will learn how to drive a majorization-minimization (MM) algorithm, which has interative updates
$$
	v_{ik}^{(t+1)} = v_{ik}^{(t)} \frac{\sum_j x_{ij} w_{kj}^{(t)}}{\sum_j b_{ij}^{(t)} w_{kj}^{(t)}}, \quad \text{where } b_{ij}^{(t)} = \sum_k v_{ik}^{(t)} w_{kj}^{(t)},
$$
$$
	w_{kj}^{(t+1)} = w_{kj}^{(t)} \frac{\sum_i x_{ij} v_{ik}^{(t)}}{\sum_i b_{ij}^{(t+1/2)} v_{ik}^{(t)}}, \quad \text{where } b_{ij}^{(t+1/2)} = \sum_k v_{ik}^{(t+1)} w_{kj}^{(t)}.
$$
Superscript $t$ indicates iteration number. Efficiency (both speed and memory) will be the most important criterion when grading this problem.


1. Implement the algorithm with arguments $\mathbf{X}$ (data, each row is a vectorized image), rank $r$, convergence tolerance, and optional starting point.
```julia
function nnmf(
    X::Matrix, 
    r::Int;
    maxiter::Int=1000, 
    tol::eltype(X)=1e-4,
    V::Matrix{eltype(X)}=rand(size(X, 1), r),
    W::Matrix{eltype(X)}=rand(r, size(X, 2))
    )
    # implementation
end
```

0. Database 1 from the [MIT Center for Biological and Computational Learning (CBCL)](http://cbcl.mit.edu) reduces to a matrix $\mathbf{X}$ containing $m = 2,429$ gray-scale face images with $n = 19 \times 19 = 361$ pixels per face. Each image (row) is scaled to have mean and standard deviation 0.25.  
Read in the [`nnmf-2429-by-361-face.txt`](http://hua-zhou.github.io/teaching/biostatm280-2018spring/hw/hw2/nnmf-2429-by-361-face.txt) file, e.g., using [`readdlm()`](https://docs.julialang.org/en/stable/stdlib/io-network/#Base.DataFmt.readdlm-Tuple{Any,Char,Type,Char}) function, and display a couple sample images, e.g., using [ImageView.jl](https://github.com/JuliaImages/ImageView.jl) package.

0. Report the run times, using `@time`, of your function for fitting NNMF on the MIT CBCL face data set at ranks $r=10, 20, 30, 40, 50$. For ease of comparison (and grading), please start your algorithm with the provided $\mathbf{V}^{(0)}$ (first $r$ columns of [`V0.txt`](http://hua-zhou.github.io/teaching/biostatm280-2018spring/hw/hw2/V0.txt)) and $\mathbf{W}^{(0)}$ (first $r$ rows of [`W0.txt`](http://hua-zhou.github.io/teaching/biostatm280-2018spring/hw/hw2/W0.txt)) and stopping criterion
$$
	\frac{|L^{(t+1)} - L^{(t)}|}{|L^{(t)}| + 1} \le 10^{-4}.
$$

0. Choose an $r \in \{10, 20, 30, 40, 50\}$ and start your algorithm from a different $\mathbf{V}^{(0)}$ and $\mathbf{W}^{(0)}$. Do you obtain the same objective value and $(\mathbf{V}, \mathbf{W})$? Explain what you find.

0. For the same $r$, start your algorithm from $v_{ik}^{(0)} = w_{kj}^{(0)} = 1$ for all $i,j,k$. Do you obtain the same objective value and $(\mathbf{V}, \mathbf{W})$? Explain what you find.

0. Plot the basis images (rows of $\mathbf{W}$) at rank $r=50$. What do you find?

0. (Optional) Investigate the GPU capabilities of Julia. Report the speed gain of your GPU code over CPU code at ranks $r=10, 20, 30, 40, 50$. Make sure to use the same starting point as in part 2.

In [7]:
#1.Implement the algorithm with arguments $f(x)$ (data, each row is a vectorized image), rank $r$, convergence tolerance, and optional starting point.

function nnmf(
 X::Matrix{T}, 
 r::Int;
 maxiter::Int = 1000, 
 tol::T = 1e-4,
 V::Matrix{T} = rand(T, size(X, 1), r),
 W::Matrix{T} = rand(T, r, size(X, 2))
 )  where T <: AbstractFloat

    # store V*W
    mul_VW = zeros(size(X))
    
    # initialize the denomenator for V update
    denomenator_V = zeros(size(X, 1), r)
    numerator_V = zeros(size(X, 1), r)
    
    denomenator_W = zeros(r, size(X, 2))
    numerator_W = zeros(r, size(X, 2))
    
    out_rr = zeros(r, r)
    
    # store the initial norm L
    # norm() is default to be Frobenius norm
    L = sum(abs2, X - A_mul_B!(mul_VW, V, W))
    
    for t in 1:maxiter # stop after 1000 iterations        
        # update v 
        # denomenator_V = V * W * W'
        # numerator_V = X * W' 
        V .= V .* (A_mul_Bt!(numerator_V, X, W) ./ A_mul_B!(denomenator_V, V, A_mul_Bt!(out_rr, W, W)))
        
        # update denomenator 
        # denomenator_W = V' * V * W
        # numerator_W = V' * X 
    
        # update W 
        W .= W .* At_mul_B!(numerator_W, V, X) ./ A_mul_B!(denomenator_W, At_mul_B!(out_rr, V, V), W)  
        
        if (sum(abs2, X - A_mul_B!(mul_VW, V, W)) - L) / (L + 1) < tol
            break
        end
        # update norm 
        L = sum(abs2, X - A_mul_B!(mul_VW, V, W))
    end  
    
    # Output
    return V, W, L
end

nnmf (generic function with 1 method)

#2
I'm so sorry that the ImageView package has some problem and cannot been used even it is installed on my laptop. That package is fine on my lab computer,and I plan to remotely control my lab computer to finish this question. However, I'm out for the conference for the whole week, and somehow my lab computer's connection is breaken on teamviewer. So I cannot get remote access to it. 

So I just put the codes below hope this would also help a little. 

```Julia
using ImageView

X = readdlm(download("http://hua-zhou.github.io/teaching/biostatm280-2018spring/hw/hw2/nnmf-2429-by-361-face.txt"), ' ')
   
y = reshape(X[1,:], 19, 19) 
imshow(y)

z = reshape(X[10,:], 19, 19) 
imshow(z)

t = reshape(X[100,:], 19, 19) 
imshow(t)
```

In [46]:
#3

X = readdlm(download("http://hua-zhou.github.io/teaching/biostatm280-2018spring/hw/hw2/nnmf-2429-by-361-face.txt"), ' ')
V0 = readdlm(download("http://hua-zhou.github.io/teaching/biostatm280-2018spring/hw/hw2/V0.txt"), ' ')
W0 = readdlm(download("http://hua-zhou.github.io/teaching/biostatm280-2018spring/hw/hw2/W0.txt"), ' ') 
    
V = V0[:,1:10]
W = W0[1:10,:]

@time a,b,l = nnmf(X, 10, V = V, W = W, maxiter = 1000,tol = (10.0^(-4)))

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 6012k  100 6012k    0     0  1838k      0  0:00:03  0:00:03 --:--:-- 1839k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  948k  100  948k    0     0  1169k      0 --:--:-- --:--:-- --:--:-- 1171k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  140k  100  140k    0     0   888k      0 --:--:-- --:--:-- --:--:--  897k


  0.130175 seconds (38 allocations: 20.498 MiB, 84.61% gc time)


([0.0559305 0.0183831 … 0.043667 0.0548117; 0.0276465 0.0622739 … 0.0123276 0.0751016; … ; 0.0791935 0.0612248 … 0.0124952 0.0249331; 0.105079 0.0948941 … 0.0828223 0.0490528], [0.218898 0.165759 … 0.0978059 0.0787535; 0.269681 0.13409 … 0.238128 0.233584; … ; 0.307447 0.171016 … 0.498192 0.302671; 0.134625 0.307717 … 0.0526499 0.0109902], 4.89877203878947e6)

In [45]:
X = readdlm(download("http://hua-zhou.github.io/teaching/biostatm280-2018spring/hw/hw2/nnmf-2429-by-361-face.txt"), ' ')
V0 = readdlm(download("http://hua-zhou.github.io/teaching/biostatm280-2018spring/hw/hw2/V0.txt"), ' ')
W0 = readdlm(download("http://hua-zhou.github.io/teaching/biostatm280-2018spring/hw/hw2/W0.txt"), ' ') 
 
V = V0[:,1:20]
W = W0[1:20,:]

@time a,b,l = nnmf(X, 20, V = V, W = W, maxiter = 1000, tol = (10.0^(-4)))

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 6012k  100 6012k    0     0  1469k      0  0:00:04  0:00:04 --:--:-- 1455k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  948k  100  948k    0     0  1344k      0 --:--:-- --:--:-- --:--:-- 1347k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  140k  100  140k    0     0  1514k      0 --:--:-- --:--:-- --:--:-- 1548k


  0.067288 seconds (35 allocations: 20.926 MiB, 64.09% gc time)


([0.024218 0.00787313 … 0.000720707 0.0160358; 0.0159894 0.0363968 … 0.0222463 0.0545939; … ; 0.0410688 0.0316813 … 0.0220709 0.0414581; 0.0635188 0.0574851 … 0.0171236 0.0163045], [0.216799 0.145065 … 0.0829008 0.0775946; 0.268986 0.116942 … 0.203839 0.233537; … ; 0.296339 0.0227248 … 0.187299 0.0612688; 0.0967789 0.345644 … 0.536204 0.503344], 2.0630806540948827e7)

In [44]:
X = readdlm(download("http://hua-zhou.github.io/teaching/biostatm280-2018spring/hw/hw2/nnmf-2429-by-361-face.txt"), ' ')
V0 = readdlm(download("http://hua-zhou.github.io/teaching/biostatm280-2018spring/hw/hw2/V0.txt"), ' ')
W0 = readdlm(download("http://hua-zhou.github.io/teaching/biostatm280-2018spring/hw/hw2/W0.txt"), ' ') 

V = V0[:,1:30]
W = W0[1:30,:]
@time a,b,l = nnmf(X, 30, V = V, W = W, maxiter = 1000, tol = (10.0^(-4)))

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 6012k  100 6012k    0     0  1733k      0  0:00:03  0:00:03 --:--:-- 1734k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  948k  100  948k    0     0   305k      0  0:00:03  0:00:03 --:--:--  306k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  140k  100  140k    0     0   570k      0 --:--:-- --:--:-- --:--:--  575k


  0.137280 seconds (38 allocations: 21.355 MiB, 83.51% gc time)


([0.0135587 0.004384 … 0.0151829 0.0162829; 0.00918104 0.0209885 … 0.0350784 0.0263492; … ; 0.0315491 0.0242566 … 0.0101406 0.00860908; 0.0414705 0.03744 … 0.00927035 0.0262603], [0.201765 0.140445 … 0.0766046 0.0700937; 0.251675 0.113174 … 0.189349 0.212177; … ; 0.211826 0.331688 … 0.449553 0.373384; 0.0979399 0.388695 … 0.0256619 0.312365], 4.770271077631289e7)

In [42]:
V = V0[:,1:40]
W = W0[1:40,:]
@time a,b,l = nnmf(X, 40, V = V, W = W, maxiter = 1000, tol = (10.0^(-4)))

  0.028852 seconds (29 allocations: 21.786 MiB, 13.49% gc time)


([0.00970053 0.00314054 … 0.0173318 0.0143655; 0.00617296 0.0141763 … 0.0146756 0.018463; … ; 0.0229953 0.017684 … 0.0216726 0.0215596; 0.0305126 0.0275719 … 0.0256064 0.0176803], [0.20265 0.148768 … 0.0734951 0.0657021; 0.253248 0.119951 … 0.182055 0.199211; … ; 0.112496 0.193331 … 0.309279 0.335764; 0.350611 0.351016 … 0.181253 0.371502], 8.650263890125868e7)

In [43]:
V = V0[:,1:50]
W = W0[1:50,:]
@time a,b,l = nnmf(X, 50, V = V, W = W, maxiter = 1000, tol = (10.0^(-4)))

  0.032421 seconds (30 allocations: 22.219 MiB, 11.27% gc time)


([0.00768694 0.00249032 … 0.00186019 0.00486848; 0.00501134 0.0115347 … 0.0101277 0.0107518; … ; 0.0190672 0.0146558 … 0.000336307 0.0141485; 0.0258745 0.0233835 … 0.0129329 0.0184881], [0.205448 0.148353 … 0.0746233 0.0662169; 0.257454 0.119838 … 0.184986 0.201132; … ; 0.192853 0.0724599 … 0.0270836 0.0234918; 0.0101171 0.376914 … 0.0870613 0.343446], 1.3594030450099576e8)

In [8]:
#4 choose r=10
X = readdlm(download("http://hua-zhou.github.io/teaching/biostatm280-2018spring/hw/hw2/nnmf-2429-by-361-face.txt"), ' ')
V0 = readdlm(download("http://hua-zhou.github.io/teaching/biostatm280-2018spring/hw/hw2/V0.txt"), ' ')
W0 = readdlm(download("http://hua-zhou.github.io/teaching/biostatm280-2018spring/hw/hw2/W0.txt"), ' ') 

V2 = V0[:,21:30]
W2 = W0[21:30,:]

c,d,m = nnmf(X, 10, V = V2, W = W2, maxiter = 1000,tol = (10.0^(-4)))
m

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 6012k  100 6012k    0     0   106k      0  0:00:56  0:00:56 --:--:--  111k   0  0:00:56  0:00:17  0:00:39 77984
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  948k  100  948k    0     0  96429      0  0:00:10  0:00:10 --:--:-- 960170  0:00:08  0:00:02  0:00:06  108k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  140k  100  140k    0     0   217k      0 --:--:-- --:--:-- --:--:--  218k


4.930143185315929e6

In [11]:
c

2429×10 Array{Float64,2}:
 0.0623597   0.00741706  0.0946621   …  0.0389711    0.033944   0.036367  
 0.00469192  0.0236487   0.041062       0.0777034    0.0804486  0.0606601 
 0.0942616   0.0147057   0.0418586      0.0186171    0.0700813  0.0500642 
 0.00235497  0.080348    0.0888608      0.030896     0.0105625  0.0770044 
 0.0073622   0.0911691   0.0849843      0.0745546    0.057464   0.0123397 
 0.0124086   0.0228995   0.0843452   …  0.0691425    0.0401416  0.0842555 
 0.0885547   0.0119347   0.0745628      0.00682047   0.0848607  0.0193776 
 0.0042406   0.0296709   0.0862552      0.0348874    0.0649731  0.0810538 
 0.0877578   0.0230472   0.0364834      0.0285422    0.0502739  0.0589645 
 0.047931    0.0906606   0.0528829      0.0308751    0.0817628  0.0651808 
 0.0277208   0.00741117  0.0828126   …  0.0608526    0.0321082  0.0563332 
 0.0789332   0.0779629   0.0512503      0.0661303    0.0448648  0.0734431 
 0.0117367   0.00121213  0.0205782      0.0549781    0.0372082  0.0889348 

In [12]:
d

10×361 Array{Float64,2}:
 0.106725   0.0276734  0.0603664  …  0.151393  0.406281   0.0766665
 0.283774   0.059007   0.145787      0.291971  0.500502   0.332068 
 0.260809   0.0783154  0.315654      0.646016  0.193381   0.329754 
 0.307976   0.380143   0.413718      0.03776   0.24966    0.0967862
 0.0770194  0.247832   0.867694      0.514348  0.46184    0.28549  
 0.283686   0.41489    0.102676   …  0.158197  0.429525   0.41643  
 0.175227   0.150964   0.0274215     0.305717  0.239838   0.0952522
 0.149568   0.336457   0.165794      0.25977   0.0385769  0.0650312
 0.193922   0.322569   0.0545961     0.603009  0.40677    0.324737 
 0.0906957  0.37656    0.439006      0.37622   0.0240529  0.274723 

Find the result is different from before, which probably due to the result is trapped in the local minimum.

5.For the same $r$, start your algorithm from $v_{ik}^{(0)} = w_{kj}^{(0)} = 1$ for all $i,j,k$. Do you obtain the same objective value and $(\mathbf{V}, \mathbf{W})$? Explain what you find.

In [15]:
#5 choose r=10

V3 = ones(size(X, 1), 10)
W3 = ones(10, size(X, 2))

e,f,n = nnmf(X, 10, V = V3, W = W3, maxiter = 1000,tol = (10.0^(-4)))

([0.0264996 0.0264996 … 0.0264996 0.0264996; 0.0264415 0.0264415 … 0.0264415 0.0264415; … ; 0.0263482 0.0263482 … 0.0263482 0.0263482; 0.0277189 0.0277189 … 0.0277189 0.0277189], [0.371818 0.461966 … 0.571488 0.438445; 0.371818 0.461966 … 0.571488 0.438445; … ; 0.371818 0.461966 … 0.571488 0.438445; 0.371818 0.461966 … 0.571488 0.438445], 8.305735921998502e7)

Find the result is different from before, which probably due to the result is trapped in the local minimum.

#6.

Similarly as the 2nd question, I don't have access to this package on my laptop while I'm out for conference for this whole week. So I just put the codes below hope this would also help a little. 

```Julia
v,w = nnmf(X,50,maxiter = 1000,tol = (10.0^(-4)))
for r in 1:50
   imshow(w[r,:])
end
```